In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import cv2

/home/wislab/anaconda3/envs/nerf-pytorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:


near = 1
#↓y →z

arr = np.array([[[ 0,  1,  2],
        [ 3,  4,  5],
        [ 6,  7,  8]],
 
       [[ 9,  10, 11],
        [12, 13, 14],
        [15, 16, 17]],
 
       [[18, 19, 20],
        [21, 22, 23],
        [24, 25, 26]]])

In [3]:
arr[...,2]

array([[ 2,  5,  8],
       [11, 14, 17],
       [20, 23, 26]])

In [4]:
import imageio
imgs = imageio.imread('data/nerf_synthetic/lego/train/r_0.png')

/tmp/ipykernel_417367/3519430871.py:2: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  imgs = imageio.imread('data/nerf_synthetic/lego/train/r_0.png')


In [5]:
imgs.shape

(800, 800, 4)

In [6]:
imgs = (np.array(imgs) / 255.).astype(np.float32) 

In [7]:
imgs.shape

(800, 800, 4)

In [8]:
W = 400
H = 400
imgs = cv2.resize(imgs, (W, H), interpolation=cv2.INTER_AREA)
imgs.shape

(400, 400, 4)

In [9]:
imgs

array([[[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       ...,

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.]],

       [[0., 0., 0., 0.],
        [0., 0., 0., 0.],
        [0., 0., 0., 0.],
        ...,
        [0., 0., 0., 0.],
        [0., 0.

In [10]:
images = imgs[...,:3]*imgs[...,-1:] + (1.-imgs[...,-1:])
images.shape
images 

array([[[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       ...,

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]],

       [[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]]], dtype=float32)

In [11]:
input_ch = 63
skips = [4]
D = 8
W = 256
pts_linears = nn.ModuleList(
            [nn.Linear(input_ch, W)] + [nn.Linear(W, W) if i not in skips else nn.Linear(W + input_ch, W) for i in range(D-1)])
pts_linears

ModuleList(
  (0): Linear(in_features=63, out_features=256, bias=True)
  (1): Linear(in_features=256, out_features=256, bias=True)
  (2): Linear(in_features=256, out_features=256, bias=True)
  (3): Linear(in_features=256, out_features=256, bias=True)
  (4): Linear(in_features=256, out_features=256, bias=True)
  (5): Linear(in_features=319, out_features=256, bias=True)
  (6): Linear(in_features=256, out_features=256, bias=True)
  (7): Linear(in_features=256, out_features=256, bias=True)
)

In [12]:
input_ch_views = 27
views_linears = nn.ModuleList([nn.Linear(input_ch_views + W, W//2)])
views_linears

ModuleList(
  (0): Linear(in_features=283, out_features=128, bias=True)
)

In [13]:
feature_linear = nn.Linear(W, W) # 额外的特征层 用于计算 alpha
alpha_linear = nn.Linear(W, 1)   # alpha 输出层
rgb_linear = nn.Linear(W//2, 3)  # RGB 输出层
feature_linear, alpha_linear, rgb_linear

(Linear(in_features=256, out_features=256, bias=True),
 Linear(in_features=256, out_features=1, bias=True),
 Linear(in_features=128, out_features=3, bias=True))

In [14]:
multires = 10
embed_kwargs = {
            'include_input' : True,         # 为真，最终的编码结果包含原始坐标
            'input_dims' : 3,               # 输入数据的维度 
            'max_freq_log2' : multires-1,   
            'num_freqs' : multires,         # 位置编码的频率数量，论文中编码公式中的L
            'log_sampling' : True,
            'periodic_fns' : [torch.sin, torch.cos],
}

In [15]:
embed_fns = []                  # 存储编码函数的列表
d = embed_kwargs['input_dims']   # 输入数据的维度
out_dim = 0                     # 编码后的总维度
if embed_kwargs['include_input']:    # 为真，最终的编码结果包含原始坐标
    embed_fns.append(lambda x : x)  # 把一个不对数据做出改变的匿名函数添加到列表中
    out_dim += d
    
max_freq = embed_kwargs['max_freq_log2'] # 位置编码的最大频率 L-1 （0-9）
N_freqs = embed_kwargs['num_freqs']      # 位置编码的频率数量，论文中编码公式中的L

if embed_kwargs['log_sampling']:       # 正弦和余弦函数的频率值 A 
    freq_bands = 2.**torch.linspace(0., max_freq, steps=N_freqs)    # 指数增长
else:
    freq_bands = torch.linspace(2.**0., 2.**max_freq, steps=N_freqs)    # 线性增长
    
'''
位置编码公式 [sin, cos]  sin(2^0 * Πx), cos(2^0 * Πx).
sin(x * freq), cos(x * freq)
x: 坐标(x, y, z), freq: 2^L (1, 2, 4, 8, 16, 32, 64, 128, 256, 512) L -> (0-9)
'''
for freq in freq_bands:
    for p_fn in embed_kwargs['periodic_fns']:
        embed_fns.append(lambda x, p_fn=p_fn, freq=freq : p_fn(x * freq))
        out_dim += d
            
embed_fns = embed_fns
out_dim = out_dim
embed_fns, out_dim

([<function __main__.<lambda>(x)>,
  <function __main__.<lambda>(x, p_fn=<built-in method sin of type object at 0x7f8bc87485e0>, freq=tensor(1.))>,
  <function __main__.<lambda>(x, p_fn=<built-in method cos of type object at 0x7f8bc87485e0>, freq=tensor(1.))>,
  <function __main__.<lambda>(x, p_fn=<built-in method sin of type object at 0x7f8bc87485e0>, freq=tensor(2.))>,
  <function __main__.<lambda>(x, p_fn=<built-in method cos of type object at 0x7f8bc87485e0>, freq=tensor(2.))>,
  <function __main__.<lambda>(x, p_fn=<built-in method sin of type object at 0x7f8bc87485e0>, freq=tensor(4.))>,
  <function __main__.<lambda>(x, p_fn=<built-in method cos of type object at 0x7f8bc87485e0>, freq=tensor(4.))>,
  <function __main__.<lambda>(x, p_fn=<built-in method sin of type object at 0x7f8bc87485e0>, freq=tensor(8.))>,
  <function __main__.<lambda>(x, p_fn=<built-in method cos of type object at 0x7f8bc87485e0>, freq=tensor(8.))>,
  <function __main__.<lambda>(x, p_fn=<built-in method sin of

In [16]:
def embed(inputs):
    # 对输入数据进行位置编码，返回编码结果
    return torch.cat([fn(inputs) for fn in embed_fns], -1)

embed_1 = lambda x : embed(x)
embed_1

<function __main__.<lambda>(x)>

In [17]:
inputs = torch.from_numpy(images)
inputs_flat = torch.reshape(inputs, [-1, inputs.shape[-1]])
inputs_flat.shape

torch.Size([160000, 3])

In [18]:
inputs[0]

tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.],
        ...,
        [1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])

In [19]:
inputs_flat[0]

tensor([1., 1., 1.])

In [20]:
embedded = embed_1(inputs_flat)
embedded.shape

torch.Size([160000, 63])

In [21]:
embedded

tensor([[ 1.0000,  1.0000,  1.0000,  ..., -0.9968, -0.9968, -0.9968],
        [ 1.0000,  1.0000,  1.0000,  ..., -0.9968, -0.9968, -0.9968],
        [ 1.0000,  1.0000,  1.0000,  ..., -0.9968, -0.9968, -0.9968],
        ...,
        [ 1.0000,  1.0000,  1.0000,  ..., -0.9968, -0.9968, -0.9968],
        [ 1.0000,  1.0000,  1.0000,  ..., -0.9968, -0.9968, -0.9968],
        [ 1.0000,  1.0000,  1.0000,  ..., -0.9968, -0.9968, -0.9968]])

In [22]:
x = np.linspace(-5, 0, 5)
y = np.linspace(0, 5, 5)
x, y

(array([-5.  , -3.75, -2.5 , -1.25,  0.  ]),
 array([0.  , 1.25, 2.5 , 3.75, 5.  ]))

In [23]:
X, Y = np.meshgrid(x, y)
X

array([[-5.  , -3.75, -2.5 , -1.25,  0.  ],
       [-5.  , -3.75, -2.5 , -1.25,  0.  ],
       [-5.  , -3.75, -2.5 , -1.25,  0.  ],
       [-5.  , -3.75, -2.5 , -1.25,  0.  ],
       [-5.  , -3.75, -2.5 , -1.25,  0.  ]])

In [24]:
Y

array([[0.  , 0.  , 0.  , 0.  , 0.  ],
       [1.25, 1.25, 1.25, 1.25, 1.25],
       [2.5 , 2.5 , 2.5 , 2.5 , 2.5 ],
       [3.75, 3.75, 3.75, 3.75, 3.75],
       [5.  , 5.  , 5.  , 5.  , 5.  ]])

In [25]:
i, j = np.meshgrid(np.arange(W, dtype=np.float32), np.arange(H, dtype=np.float32), indexing='xy')

In [26]:
i, j

(array([[  0.,   1.,   2., ..., 253., 254., 255.],
        [  0.,   1.,   2., ..., 253., 254., 255.],
        [  0.,   1.,   2., ..., 253., 254., 255.],
        ...,
        [  0.,   1.,   2., ..., 253., 254., 255.],
        [  0.,   1.,   2., ..., 253., 254., 255.],
        [  0.,   1.,   2., ..., 253., 254., 255.]], dtype=float32),
 array([[  0.,   0.,   0., ...,   0.,   0.,   0.],
        [  1.,   1.,   1., ...,   1.,   1.,   1.],
        [  2.,   2.,   2., ...,   2.,   2.,   2.],
        ...,
        [397., 397., 397., ..., 397., 397., 397.],
        [398., 398., 398., ..., 398., 398., 398.],
        [399., 399., 399., ..., 399., 399., 399.]], dtype=float32))

In [27]:
focal = 1
K = np.array([
    [focal, 0, 0.5*W],
    [0, focal, 0.5*H],
    [0, 0, 1]
])

In [28]:
dirs = np.stack([(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -np.ones_like(i)], -1)
dirs

array([[[-128.,  200.,   -1.],
        [-127.,  200.,   -1.],
        [-126.,  200.,   -1.],
        ...,
        [ 125.,  200.,   -1.],
        [ 126.,  200.,   -1.],
        [ 127.,  200.,   -1.]],

       [[-128.,  199.,   -1.],
        [-127.,  199.,   -1.],
        [-126.,  199.,   -1.],
        ...,
        [ 125.,  199.,   -1.],
        [ 126.,  199.,   -1.],
        [ 127.,  199.,   -1.]],

       [[-128.,  198.,   -1.],
        [-127.,  198.,   -1.],
        [-126.,  198.,   -1.],
        ...,
        [ 125.,  198.,   -1.],
        [ 126.,  198.,   -1.],
        [ 127.,  198.,   -1.]],

       ...,

       [[-128., -197.,   -1.],
        [-127., -197.,   -1.],
        [-126., -197.,   -1.],
        ...,
        [ 125., -197.,   -1.],
        [ 126., -197.,   -1.],
        [ 127., -197.,   -1.]],

       [[-128., -198.,   -1.],
        [-127., -198.,   -1.],
        [-126., -198.,   -1.],
        ...,
        [ 125., -198.,   -1.],
        [ 126., -198.,   -1.],
        [ 127.,

In [29]:
dirs = np.stack([(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -np.ones_like(i)])
dirs.shape

(3, 400, 256)

In [30]:
[(i-K[0][2])/K[0][0], -(j-K[1][2])/K[1][1], -np.ones_like(i)]

[array([[-128., -127., -126., ...,  125.,  126.,  127.],
        [-128., -127., -126., ...,  125.,  126.,  127.],
        [-128., -127., -126., ...,  125.,  126.,  127.],
        ...,
        [-128., -127., -126., ...,  125.,  126.,  127.],
        [-128., -127., -126., ...,  125.,  126.,  127.],
        [-128., -127., -126., ...,  125.,  126.,  127.]], dtype=float32),
 array([[ 200.,  200.,  200., ...,  200.,  200.,  200.],
        [ 199.,  199.,  199., ...,  199.,  199.,  199.],
        [ 198.,  198.,  198., ...,  198.,  198.,  198.],
        ...,
        [-197., -197., -197., ..., -197., -197., -197.],
        [-198., -198., -198., ..., -198., -198., -198.],
        [-199., -199., -199., ..., -199., -199., -199.]], dtype=float32),
 array([[-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        ...,
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1., -1., ..., -1., -1., -1.],
        [-1., -1.,

In [31]:
import numpy as np

a = np.array([[1, 2, 3], 
              [4, 5, 6]])
b = np.array([[7, 8, 9],
              [10, 11, 12]])
c = np.stack([a, b], 2)

c


array([[[ 1,  7],
        [ 2,  8],
        [ 3,  9]],

       [[ 4, 10],
        [ 5, 11],
        [ 6, 12]]])